### 1. Pengenalan Spark DataFrames
Spark DataFrame menyediakan struktur data yang optimal dengan operasi yang dioptimalkan untuk pemrosesan data besar, yang sangat mirip dengan DataFrame di Pandas atau di RDBMS.

- **Tugas 1**: Buat DataFrame sederhana di Spark dan eksplorasi beberapa fungsi dasar yang tersedia.

In [65]:
# Contoh membuat DataFrame sederhana dan operasi dasar
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('HandsOnPertemuan3').getOrCreate()

data = [('James', 'Sales', 3000),
        ('Michael', 'Sales', 4600),
        ('Robert', 'Sales', 4100),
        ('Maria', 'Finance', 3000)]
columns = ['EmployeeName', 'Department', 'Salary']

df = spark.createDataFrame(data, schema=columns)
df.show()

+------------+----------+------+
|EmployeeName|Department|Salary|
+------------+----------+------+
|       James|     Sales|  3000|
|     Michael|     Sales|  4600|
|      Robert|     Sales|  4100|
|       Maria|   Finance|  3000|
+------------+----------+------+



### 2. Transformasi Dasar dengan DataFrames
Pemrosesan data meliputi transformasi seperti filtering, selections, dan aggregations. Spark menyediakan cara efisien untuk melaksanakan operasi ini.

- **Tugas 2**: Gunakan operasi filter, select, groupBy untuk mengekstrak informasi dari data, serta lakukan agregasi data untuk mendapatkan insight tentang dataset menggunakan perintah seperti mean, max, sum.

In [ ]:
# Contoh operasi transformasi DataFrame
select('EmployeeName', 'Salary').show()
filter(df['Salary'] > 3000).show()
groupBy('Department').avg('Salary').show()

In [76]:
#Select kolom tertentu
print("Data Kolom EmployeeName dan Salary")
df.select('EmployeeName', 'Salary').show()

#Filter gaji lebih dari 3000
print("Salary above 3000")
df.filter(df['Salary'] > 3000).show()

#GroupBy Department dan hitung rata-rata gaji
print("Mean Salary per Department")
df.groupBy('Department').avg('Salary').show()

#GroupBy Department dan hitung nilai maksimum gaji
print("Salary Max per Department")
df.groupBy('Department').max('Salary').show()

#GroupBy Department dan hitung total gaji
print("Total Salary per Department")
df.groupBy('Department').sum('Salary').show()


Data Kolom EmployeeName dan Salary
+------------+------+
|EmployeeName|Salary|
+------------+------+
|       James|  3000|
|     Michael|  4600|
|      Robert|  4100|
|       Maria|  3000|
+------------+------+

Salary above 3000
+------------+----------+------+
|EmployeeName|Department|Salary|
+------------+----------+------+
|     Michael|     Sales|  4600|
|      Robert|     Sales|  4100|
+------------+----------+------+

Mean Salary per Department
+----------+-----------+
|Department|avg(Salary)|
+----------+-----------+
|     Sales|     3900.0|
|   Finance|     3000.0|
+----------+-----------+

Salary Max per Department
+----------+-----------+
|Department|max(Salary)|
+----------+-----------+
|     Sales|       4600|
|   Finance|       3000|
+----------+-----------+

Total Salary per Department
+----------+-----------+
|Department|sum(Salary)|
+----------+-----------+
|     Sales|      11700|
|   Finance|       3000|
+----------+-----------+



### 3. Bekerja dengan Tipe Data Kompleks
Spark mendukung tipe data yang kompleks seperti maps, arrays, dan structs yang memungkinkan operasi yang lebih kompleks pada dataset yang kompleks.

- **Tugas 3**: Eksplorasi bagaimana mengolah tipe data kompleks dalam Spark DataFrames.

In [ ]:
# Contoh manipulasi tipe data kompleks
df.withColumn('SalaryBonus', df['Salary'] * 0.1).show()
df.withColumn('TotalCompensation', df['Salary'] + df['SalaryBonus']).show()

In [127]:
# Contoh manipulasi tipe data kompleks
from pyspark.sql.functions import explode
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, MapType

# Tambah kolom Array Skill
data_array = [
    ('James', 'Sales', 3000, ["Excel", "PowerPoint"]),
    ('Michael', 'Sales', 4600, ["Python", "SQL"]),
    ('Robert', 'Sales', 4100, ["Java", "Scala"]),
    ('Maria', 'Finance', 3000, ["Accounting", "Excel"])
]

columns_array = ['EmployeeName', 'Department', 'Salary', 'Skills']
df_array = spark.createDataFrame(data_array, schema=columns_array)

print("Data dengan Skills")
df_array.show(truncate=False)

# Explode array ke baris terpisah
print("Data EmployeeName dan Skill")
df_array.select("EmployeeName", explode("Skills").alias("Skill")).show()

#Tambah kolom Struct addres
data_struct = [
    ('James', 'Sales', 3000, ("Gotham", 1231)),
    ('Michael', 'Sales', 4600, ("Untid", 1232)),
    ('Robert', 'Sales', 4100, ("Bikini Bottom", 1233)),
    ('Maria', 'Finance', 3000, ("Destian", 1234))
]

schema_struct = StructType([
    StructField("EmployeeName", StringType(), True),
    StructField("Department", StringType(), True),
    StructField("Salary", IntegerType(), True),
    StructField("Address", StructType([
        StructField("City", StringType(), True),
        StructField("Zipcode", IntegerType(), True)
    ]))
])

df_struct = spark.createDataFrame(data_struct, schema=schema_struct)

print("Data dengan Addres")
df_struct.show(truncate=False)

# Akses field nested
print("Data EmployeeName, City, dan Zipcode")
df_struct.select("EmployeeName", "Address.City", "Address.Zipcode").show()

Data dengan Skills
+------------+----------+------+-------------------+
|EmployeeName|Department|Salary|Skills             |
+------------+----------+------+-------------------+
|James       |Sales     |3000  |[Excel, PowerPoint]|
|Michael     |Sales     |4600  |[Python, SQL]      |
|Robert      |Sales     |4100  |[Java, Scala]      |
|Maria       |Finance   |3000  |[Accounting, Excel]|
+------------+----------+------+-------------------+

Data EmployeeName dan Skill
+------------+----------+
|EmployeeName|     Skill|
+------------+----------+
|       James|     Excel|
|       James|PowerPoint|
|     Michael|    Python|
|     Michael|       SQL|
|      Robert|      Java|
|      Robert|     Scala|
|       Maria|Accounting|
|       Maria|     Excel|
+------------+----------+

Data dengan Addres
+------------+----------+------+---------------------+
|EmployeeName|Department|Salary|Address              |
+------------+----------+------+---------------------+
|James       |Sales     |3000  

### 4. Operasi Data Lanjutan
Menggunakan Spark untuk operasi lanjutan seperti window functions, user-defined functions (UDFs), dan mengoptimalkan query.

- **Tugas 4**: Implementasikan window function untuk menghitung running totals atau rangkings.

In [128]:
# Contoh menggunakan window functions
from pyspark.sql.window import Window
from pyspark.sql import functions as F

windowSpec = Window.partitionBy('Department').orderBy('Salary')
df.withColumn('Rank', F.rank().over(windowSpec)).show()

+------------+----------+------+----+
|EmployeeName|Department|Salary|Rank|
+------------+----------+------+----+
|       Maria|   Finance|  3000|   1|
|       James|     Sales|  3000|   1|
|      Robert|     Sales|  4100|   2|
|     Michael|     Sales|  4600|   3|
+------------+----------+------+----+



In [129]:
# Import library
from pyspark.sql.window import Window
from pyspark.sql import functions as F

# WindowSpec: kelompokkan per Department, urutkan berdasarkan Salary
windowSpec = Window.partitionBy('Department').orderBy('Salary')

# Contoh menggunakan window functions untuk ranking
df_with_rank = df.withColumn('Rank', F.rank().over(windowSpec))
print("Penggunaan window functions untuk ranking")
df_with_rank.show()

# Contoh menggunakan window functions untuk running total
df_running = df.withColumn('RunningTotal', F.sum('Salary').over(windowSpec))
print("Penggunaan window functions untuk running total")
df_running.show()

Penggunaan window functions untuk ranking
+------------+----------+------+----+
|EmployeeName|Department|Salary|Rank|
+------------+----------+------+----+
|       Maria|   Finance|  3000|   1|
|       James|     Sales|  3000|   1|
|      Robert|     Sales|  4100|   2|
|     Michael|     Sales|  4600|   3|
+------------+----------+------+----+

Penggunaan window functions untuk running total
+------------+----------+------+------------+
|EmployeeName|Department|Salary|RunningTotal|
+------------+----------+------+------------+
|       Maria|   Finance|  3000|        3000|
|       James|     Sales|  3000|        3000|
|      Robert|     Sales|  4100|        7100|
|     Michael|     Sales|  4600|       11700|
+------------+----------+------+------------+



### 5. Kesimpulan dan Eksplorasi Lebih Lanjut
Review apa yang telah dipelajari tentang pemrosesan data menggunakan Spark dan eksplorasi teknik lebih lanjut untuk mengoptimalkan pemrosesan data Anda.
<br>**Tugas 5**:
- Unduh dataset besar dari [Kaggle](https://www.kaggle.com/) atau sumber lainnya.
- Input data csv yang telah di download, kemudian load dan simpan data ke dalam pyspark.
- Setelah data berhasil di load menggunakan pyspark, lakukan manipulasi data untuk memperoleh informasi yang dibutuhkan

In [124]:
from pyspark.sql import SparkSession

# Inisialisasi SparkSession
spark = SparkSession.builder.appName("DPR-RI").getOrCreate()

# Load dataset dari CSV
df_dpr = spark.read.csv("Anggota DPR RI 2024-2029.csv", header=True, inferSchema=True)

# Tampilkan beberapa baris awal
df_dpr.show()

# Cek skema dataset
df_dpr.printSchema()

+-----------+----------------+--------------------+-------------+-----------+--------+--------------------+--------------+------------------+
|No. Anggota|Daerah pemilihan|                Nama|Jenis Kelamin|     Komisi|  Fraksi|             Jabatan|      Provinsi|           Koalisi|
+-----------+----------------+--------------------+-------------+-----------+--------+--------------------+--------------+------------------+
|          1|          Aceh I|H. Irmawan S.Sos....|    Laki-Laki|   Komisi V|     PKB|                   -|          Aceh|           Oposisi|
|        155|          Aceh I|Jamaluddin Idham ...|    Laki-Laki|Komisi XIII|   PDI-P|                   -|          Aceh|           Oposisi|
|        264|          Aceh I|Drs. H. T. Zulkar...|    Laki-Laki| Komisi VII|  Golkar|                   -|          Aceh|Koalisi Pemerintah|
|        366|          Aceh I|H. Muslim Ayub S....|    Laki-Laki|Komisi XIII|  NasDem|                   -|          Aceh|           Oposisi|
|     

In [152]:
# Contoh: pilih anggota dari Fraksi PDIP
df_dpr.filter(df_dpr.Fraksi == "PDI-P").show()

# Contoh: pilih anggota dari Provinsi Jawa Tengah
df_dpr.filter(df_dpr.Provinsi == "Jawa Tengah").show()


+-----------+--------------------+--------------------+-------------+-----------+------+--------------------+--------------------+-------+
|No. Anggota|    Daerah pemilihan|                Nama|Jenis Kelamin|     Komisi|Fraksi|             Jabatan|            Provinsi|Koalisi|
+-----------+--------------------+--------------------+-------------+-----------+------+--------------------+--------------------+-------+
|        155|              Aceh I|Jamaluddin Idham ...|    Laki-Laki|Komisi XIII| PDI-P|                   -|                Aceh|Oposisi|
|        156|    Sumatera Utara I|      Dr. Sofyan Tan|    Laki-Laki|   Komisi X| PDI-P|                   -|      Sumatera Utara|Oposisi|
|        157|    Sumatera Utara I|Prof. Dr. Yasonna...|    Laki-Laki|Komisi XIII| PDI-P|                   -|      Sumatera Utara|Oposisi|
|        158|   Sumatera Utara II|Drs. Rapidin Simb...|    Laki-Laki|Komisi XIII| PDI-P|                   -|      Sumatera Utara|Oposisi|
|        159|   Sumatera Ut

In [155]:
# Group by Fraksi
df_dpr.groupBy("Fraksi").count().show()

# Group by Provinsi
df_dpr.groupBy("Provinsi").count().show()

+--------+-----+
|  Fraksi|count|
+--------+-----+
|   PDI-P|  109|
|     PKS|   53|
|Demokrat|   44|
|     PAN|   48|
|  NasDem|   69|
|  Golkar|  102|
|     PKB|   68|
|Gerindra|   86|
+--------+-----+

+-------------------+-----+
|           Provinsi|count|
+-------------------+-----+
|  Kalimantan Tengah|    6|
|              Papua|    3|
|     Sulawesi Barat|    4|
|     Kepulauan Riau|    4|
|Nusa Tenggara Barat|   11|
|     Sulawesi Utara|    6|
|     Sumatera Utara|   30|
|        DKI Jakarta|   21|
|   Kalimantan Utara|    3|
|      Papua Selatan|    3|
|     Sumatera Barat|   14|
|   Kalimantan Barat|   12|
|   Sulawesi Selatan|   24|
|               Aceh|   13|
|        Papua Barat|    3|
|       Papua Tengah|    3|
|   Kalimantan Timur|    8|
|               Bali|    9|
|              Jambi|    8|
|       Maluku Utara|    3|
+-------------------+-----+
only showing top 20 rows



In [146]:
from pyspark.sql import functions as F

# Hitung total anggota
df_dpr.agg(F.count("*").alias("Total Anggota")).show()

# Jumlah anggota per fraksi
df_dpr.groupBy("Fraksi").agg(F.count("*").alias("Jumlah_Anggota")).show()

# Jumlah anggota per provinsi
df_dpr.groupBy("Provinsi").agg(F.count("*").alias("Jumlah_Anggota")).show()

# Jumlah anggota laki-laki & perempuan per fraksi
df_dpr.groupBy("Fraksi", "Jenis Kelamin").agg(F.count("*").alias("Jumlah")).show()

+-------------+
|Total Anggota|
+-------------+
|          579|
+-------------+

+--------+--------------+
|  Fraksi|Jumlah_Anggota|
+--------+--------------+
|   PDI-P|           109|
|     PKS|            53|
|Demokrat|            44|
|     PAN|            48|
|  NasDem|            69|
|  Golkar|           102|
|     PKB|            68|
|Gerindra|            86|
+--------+--------------+

+-------------------+--------------+
|           Provinsi|Jumlah_Anggota|
+-------------------+--------------+
|  Kalimantan Tengah|             6|
|              Papua|             3|
|     Sulawesi Barat|             4|
|     Kepulauan Riau|             4|
|Nusa Tenggara Barat|            11|
|     Sulawesi Utara|             6|
|     Sumatera Utara|            30|
|        DKI Jakarta|            21|
|   Kalimantan Utara|             3|
|      Papua Selatan|             3|
|     Sumatera Barat|            14|
|   Kalimantan Barat|            12|
|   Sulawesi Selatan|            24|
|             